1. 필요한 라이브러리, 모듈 가져오기

In [33]:
import torch
import torch.optim as optim
import torch.nn as nn
from torchtext.vocab import Vocab
# 글자 전처리를 위한 import
from torchtext.data.utils import get_tokenizer
import pandas as pd
import os
import importlib.util
import pickle

# 모듈을 동적으로 불러오는 함수
def load_module_from_path(path):
    module_name = os.path.splitext(os.path.basename(path))[0]
    spec = importlib.util.spec_from_file_location(module_name, path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return module

# 파일 경로 설정
preProcessing_path = "preProcessing/sentiment.py"
attention_path = "model_class/Attention.py"
lstm_path = "model_class/LSTMModel.py"

# 각 파일 모듈 불러오기
preProcessing = load_module_from_path(preProcessing_path)
Attention = load_module_from_path(attention_path)
LSTMModel = load_module_from_path(lstm_path)

# kaggle 넷플릭스 리뷰
data = pd.read_csv("data/netflix_reviews.csv")

# 저장된 단어집 가져오기
with open("data/vocab.pkl", "rb") as f:
    stoi = pickle.load(f)

2. 함수 불러오기 및 파이프 파인 함수 정의

In [34]:
train_reviews = preProcessing.getSentimentTrainReviews(data)
tokenizer = get_tokenizer('basic_english')
vocab = Vocab(stoi)

def text_pipeline(text):
    return [vocab[token] for token in tokenizer(text)]

def label_pipeline(label):
    return int(label)

3. 기본 LSTM 모델 정의

In [35]:
# 하이퍼파라미터 수정
VOCAB_SIZE = len(vocab)
EMBED_DIM = 64  
HIDDEN_DIM = 128  
OUTPUT_DIM = 3
NUM_EPOCHS = 100
LABEL_SMOOTHING = 0.05

model_in_path =  LSTMModel.LSTMModel(VOCAB_SIZE, EMBED_DIM, HIDDEN_DIM, OUTPUT_DIM)


# 손실 함수와 옵티마이저 정의
# 라벨 스무딩을 이용하여 모델의 일반화 성능을 향상시킴 (평점이 리뷰와 안맞는 케이스 완화)
criterion = nn.CrossEntropyLoss(label_smoothing=LABEL_SMOOTHING)
optimizer = optim.Adam(model_in_path.parameters(), lr=0.01)

4. 미리 학습된 모델 가져와 load

In [37]:
save_path = "lstm_sentiment_model_vocab.pth"
checkpoint = torch.load(save_path)
model_in_path.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

5. 예측 함수 정의

In [38]:
# 예측 함수(예시)
def predict_review(model, review):
    model.eval()
    with torch.no_grad():
        tensor_review = torch.tensor(text_pipeline(review), dtype=torch.long).unsqueeze(0)
        output = model(tensor_review)
        prediction =output.argmax(dim=1).item()
        return prediction

6. 예측 진행

In [39]:
# 새로운 리뷰에 대한 예측
new_review = "This app is good but there are some critical bugs"
predicted_score = predict_review(model_in_path, new_review)
print(f'Predicted Score: {predicted_score}')

Predicted Score: 2


In [40]:
# 새로운 리뷰에 대한 예측
new_review = "I have little disappointed about this app"
predicted_score = predict_review(model_in_path, new_review)
print(f'Predicted Score: {predicted_score}')

Predicted Score: 0


In [41]:
# 새로운 리뷰에 대한 예측
new_review = "So great"
predicted_score = predict_review(model_in_path, new_review)
print(f'Predicted Score: {predicted_score}')

Predicted Score: 2


In [42]:
new_review = "So Awful"
predicted_score = predict_review(model_in_path, new_review)
print(f'Predicted Score: {predicted_score}')

Predicted Score: 0
